In [1]:
from line_profiler import LineProfiler

In [2]:
from line_profiler import LineProfiler

# Example function
def expensive_function(n=10):
    total = 0
    for i in range(n):
        total += i
    return total

# Create a LineProfiler instance
profiler = LineProfiler()

# Add the function to the profiler
profiler.add_function(expensive_function)

# Run the function to collect profiling data
expensive_function()

# Retrieve the profiling stats as LineStats
stats = profiler.get_stats()

# Print the profiling data for 'expensive_function'
stats.timings

{('/var/folders/k9/6y59tr5d45s1nynfg_sv5zr80000gn/T/ipykernel_976/3009186789.py',
  4,
  'expensive_function'): []}

In [3]:
profiler.print_stats()

Timer unit: 1e-09 s

Total time: 0 s
File: /var/folders/k9/6y59tr5d45s1nynfg_sv5zr80000gn/T/ipykernel_976/3009186789.py
Function: expensive_function at line 4

Line #      Hits         Time  Per Hit   % Time  Line Contents
     4                                           def expensive_function(n=10):
     5                                               total = 0
     6                                               for i in range(n):
     7                                                   total += i
     8                                               return total



In [4]:
stats.timings

{('/var/folders/k9/6y59tr5d45s1nynfg_sv5zr80000gn/T/ipykernel_976/3009186789.py',
  4,
  'expensive_function'): []}

In [7]:
import sys
import io
from line_profiler import LineProfiler

# A sample function to profile
def expensive_function():
    a = 0
    for i in range(1000):
        a += i
    return a

# Create a LineProfiler instance
profiler = LineProfiler()

# Add the function to the profiler
profiler.add_function(expensive_function)

# Run the function while profiling
profiler.enable_by_count()  # Enable the profiler
expensive_function()
profiler.disable_by_count()  # Disable the profiler

# Capture the output of print_stats
output = io.StringIO()
sys.stdout = output  # Redirect stdout to capture print_stats output
profiler.print_stats()
sys.stdout = sys.__stdout__  # Reset stdout

# Now 'output' contains everything that print_stats would print
stats_output = output.getvalue()


In [11]:
stats_output

'Timer unit: 1e-09 s\n\nTotal time: 0.001937 s\nFile: /var/folders/k9/6y59tr5d45s1nynfg_sv5zr80000gn/T/ipykernel_976/4288502995.py\nFunction: expensive_function at line 6\n\nLine #      Hits         Time  Per Hit   % Time  Line Contents\n==============================================================\n     6                                           def expensive_function():\n     7         1       1000.0   1000.0      0.1      a = 0\n     8      1001     487000.0    486.5     25.1      for i in range(1000):\n     9      1000    1449000.0   1449.0     74.8          a += i\n    10         1          0.0      0.0      0.0      return a\n\n'

python_profiling.time_profiling.time_profiler.TimeProfiler